In [ ]:
%pip install transformers

In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import TFDistilBertForSequenceClassification


model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [3]:
import tensorflow as tf
import pandas as pd

In [4]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN):
    train_InputExamples = train.apply(
        lambda x: InputExample(
            guid=
            None,  # Globally unique ID for bookkeeping, unused in this case
            text_a=x[DATA_COLUMN],
            text_b=None,
            label=x[LABEL_COLUMN]),
        axis=1)

    validation_InputExamples = test.apply(
        lambda x: InputExample(
            guid=
            None,  # Globally unique ID for bookkeeping, unused in this case
            text_a=x[DATA_COLUMN],
            text_b=None,
            label=x[LABEL_COLUMN]),
        axis=1)

    return train_InputExamples, validation_InputExamples

In [5]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []  # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,  # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            # padding="longest", # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True)

        input_ids, token_type_ids, attention_mask = (
            input_dict["input_ids"], input_dict["token_type_ids"],
            input_dict['attention_mask'])

        features.append(
            InputFeatures(input_ids=input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids,
                          label=e.label))

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({
            "input_ids": tf.int32,
            "attention_mask": tf.int32,
            "token_type_ids": tf.int32
        }, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [6]:
# temp_test = pd.read_csv("../Crawled_data/labeled-daniil.csv",
#                         encoding="utf-8",
#                         sep=";")
# # temp_test.head()

# test = temp_test[["sentiment", "title"]]
# test = test.rename(columns={"sentiment": "label", "title": "data"})

# test["label"] = test["label"].replace(["positive"], "2")
# test["label"] = test["label"].replace(["neutral"], "1")
# test["label"] = test["label"].replace(["negative"], "0")

dataset = pd.read_csv("../dataset/all-data.csv", encoding="ISO-8859-1", engine="python")

dataset["label"] = dataset["label"].replace(["positive"], "2")
dataset["label"] = dataset["label"].replace(["neutral"], "1")
dataset["label"] = dataset["label"].replace(["negative"], "0")

# for _, row in dataset.iterrows():
#     if row["label"] != "0" and row["label"] != "1" and row["label"] != "2":
#         print("shit:", row["label"])

test = dataset.sample(480)

dataset = dataset.drop(test.index)

print(dataset.shape, test.shape)

# test.head()
# dataset.head()

(4366, 2) (480, 2)


In [7]:
DATA_COLUMN = "data"
LABEL_COLUMN = "label"

train_InputExamples, validation_InputExamples = convert_data_to_examples(
    dataset, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples),
                                            tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(
    list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

c:\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,
                                     epsilon=1e-08,
                                     clipnorm=1.0)
loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.constant([1., 1.,
                                                                   1.]),
                                               labels=tf.constant([1., 1.,
                                                                   1.]))
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.fit(train_data, epochs=1, validation_data=validation_data)


TypeError: Cannot convert 'passthrough' to EagerTensor of dtype float